In [ ]:
from pasteur.kedro.ipython import * # type: ignore
register_kedro() # type: ignore

%load_ext autoreload
%autoreload 2

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84


In [ ]:
import pandas as pd

TARGET = 1e9 # scale partition to 1 billion
GB_BYTES = 1e9

def mem_usage(df: pd.DataFrame):
    """ Calculates the equivalent memory usage when scaled to 1 billion rows. """
    ngb = sum(df.memory_usage(deep=True)) / GB_BYTES * (TARGET / df.shape[0])
    print(f"Number of rows: {df.shape[0]:,d}")
    print(f'Size for 1B rows: {ngb:,.2f} GB')

In [ ]:
optimised = catalog.load('mimic_billion.wrk.table').sample()
optimised.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15351224 entries, 51118402 to 59901588
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   charttime       datetime64[ns]
 1   valuenum        float32       
 2   valueuom        category      
 3   warning         bool          
 4   first_careunit  category      
 5   last_careunit   category      
 6   intime          datetime64[ns]
 7   outtime         datetime64[ns]
 8   gender          category      
 9   birth_year      datetime64[ns]
dtypes: bool(1), category(4), datetime64[ns](4), float32(1)
memory usage: 717.4 MB


In [ ]:
mem_usage(optimised)

Number of rows: 15,351,224
Size for 1B rows: 49.00 GB


In [ ]:
raw = optimised.astype(
    {
        # By default, 
        "valuenum": "float64",
        # By default, string columns become objects. Category is much more efficient.
        "valueuom": "object",
        "first_careunit": "object",
        "last_careunit": "object",
        "gender": "object",
        # By default, pandas doesn't parse dates
        "intime": "object",
        "outtime": "object",
        "birth_year": "object",
        "charttime": "object"
    }
)
mem_usage(raw)


Number of rows: 15,351,224
Size for 1B rows: 776.87 GB


In [ ]:
idx = catalog.load('mimic_billion.wrk.idx_table').sample()
idx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15351224 entries, 51118402 to 59901588
Data columns (total 16 columns):
 #   Column          Dtype
---  ------          -----
 0   intime_year     uint8
 1   intime_week     uint8
 2   intime_day      uint8
 3   intime_time     uint8
 4   outtime_week    uint8
 5   outtime_day     uint8
 6   outtime_time    uint8
 7   charttime_week  uint8
 8   charttime_day   uint8
 9   charttime_time  uint8
 10  valuenum        uint8
 11  valueuom        uint8
 12  warning         uint8
 13  first_careunit  uint8
 14  last_careunit   uint8
 15  gender          uint8
dtypes: uint8(16)
memory usage: 351.4 MB


In [ ]:
mem_usage(idx)

Number of rows: 15,351,224
Size for 1B rows: 24.00 GB


In [ ]:
idx64 = idx.astype('int64')
mem_usage(idx64)

Number of rows: 15,351,224
Size for 1B rows: 136.00 GB


In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

tmp = Path('.sizing')
!mkdir -p {tmp}

print("Orig: .csv -> .csv.gz -> .pq")
%time optimised.to_csv(tmp / "orig.csv")
%time optimised.to_csv(tmp / "orig.csv.gz")
%time optimised.to_parquet(tmp / "orig.pq")

print("idx: .csv -> .csv.gz -> .pq")
%time idx.to_csv(tmp / "idx.csv")
%time idx.to_csv(tmp / "idx.csv.gz")
%time idx.to_parquet(tmp / "idx.pq")

print("idx64: .csv -> .csv.gz -> .pq")
%time idx64.to_csv(tmp / "idx64.csv")
%time idx64.to_csv(tmp / "idx64.csv.gz")
%time idx64.to_parquet(tmp / "idx64.pq")

print("File sizes")
out = !ls -lah {tmp}
# Remove my username
print(" ".join(["username" if "@" in s else s for s in "\n".join(out).split(" ")]))

!rm -r {tmp}

Orig: .csv -> .csv.gz -> .pq
CPU times: user 2min 23s, sys: 3.64 s, total: 2min 27s
Wall time: 2min 27s
CPU times: user 5min 38s, sys: 1.23 s, total: 5min 40s
Wall time: 5min 40s
CPU times: user 5.98 s, sys: 392 ms, total: 6.37 s
Wall time: 7.18 s
idx: .csv -> .csv.gz -> .pq
CPU times: user 56.2 s, sys: 1.22 s, total: 57.4 s
Wall time: 57.4 s
CPU times: user 5min 40s, sys: 930 ms, total: 5min 41s
Wall time: 5min 46s
CPU times: user 5.74 s, sys: 473 ms, total: 6.22 s
Wall time: 8.71 s
idx64: .csv -> .csv.gz -> .pq
CPU times: user 1min 12s, sys: 1.84 s, total: 1min 13s
Wall time: 1min 25s
CPU times: user 5min 55s, sys: 1.02 s, total: 5min 56s
Wall time: 6min 25s
CPU times: user 5.16 s, sys: 332 ms, total: 5.49 s
Wall time: 6.32 s
File sizes
total 5.2G
drwxrwxr-x 2 username username 4.0K Feb  3 19:19 .
drwxrwxr-x 3 username username 4.0K Feb  3 19:19 ..
-rw-rw-r-- 1 username username 660M Feb  3 19:06 idx.csv
-rw-rw-r-- 1 username username 210M Feb  3 19:11 idx.csv.gz
-rw-rw-r-- 1 usernam

In [ ]:
multiplier = 1e9 / idx.shape[0]
print(f"Multiplier to 1B: {multiplier:,.4f}")

Multiplier to 1B: 65.1414


In [ ]:
MULT = multiplier
mult = lambda min, sec: f"{int((MULT*(min*60+sec)) // 60)}:{int((MULT*(min*60+sec)) % 60):.3f}"
# mult(2, 23)...